<a href="https://colab.research.google.com/github/ECE-Mizukami/thesis_data/blob/main/TrackingTool_6_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
elementumName = 'NH4Cl'

In [ ]:
import pathlib
import cv2
import numpy as np
import csv

#ここから輪郭抽出処理を定義（ここでは何もしない）
def contours(img):
        contours, hierarchy = cv2.findContours(threshold_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        areaTable = np.empty(0)
        if len(contours) != 0:
          for i in range(len(contours)):
           contourArea = cv2.contourArea(contours[i])
           A = int(contourArea)
           areaTable = np.append(areaTable,A)
          area = np.amax(areaTable)
          frameContour = np.argmax(areaTable)
          #contours = np.array(contours)
          if area > 0.05:
            x_max = np.amax(contours[frameContour].T[0, 0])
            x_min = np.amin(contours[frameContour].T[0, 0])
            y_max = np.amax(contours[frameContour].T[1, 0])
            y_min = np.amin(contours[frameContour].T[1, 0])
            contoursNum = len(contours)
          else:
            x_max = 0
            x_min = 0
            y_max = 0
            y_min = 0
            contoursNum = 0
          return x_max,x_min,y_max,y_min,area,contoursNum
        else:
          return 0,0,0,0,0,0

folderList = []
p_temp = pathlib.Path('/content/drive/MyDrive/動画データ').glob(str(elementumName)+'*')
for p in p_temp:
  folderList.append(p.name)
for i in range(len(folderList)):
#for i in range(0,2):
  folderPath =  ('/content/drive/MyDrive/動画データ/'+str(folderList[i]))
  fileList = []
  q_temp = pathlib.Path(str(folderPath)).glob('*.mp4')
  print(folderList)
  for q in q_temp:
      fileList.append(q.name)
  for j in range(len(fileList)):
    filePath = ('/content/drive/MyDrive/動画データ/'+str(folderList[i])+'/'+str(fileList[j]))
    fileName = filePath[:-4]
    cap = cv2.VideoCapture(filePath)
    if not cap.isOpened():
      print("missed")

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))
    m = 0
    integral_threshould = 0
    while True: #とりあえずすべてのフレームで閾値を求め、その平均を動画全体の閾値とする
        is_image,frame_img = cap.read()#フレームの読み込み
        if is_image:
          #ここから二値化処理
          gray = cv2.cvtColor(frame_img, cv2.COLOR_RGB2GRAY)
          threshold_img = gray.copy()
          point_threshould, dst = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE) #トライアングルアルゴリズムは神、大津は微妙
          #print(point_threshould,integral_threshould) #デバック用。各コマの閾値が見える。
        else:
            break
        threshold_img[gray < point_threshould] = 0
        threshold_img[gray >= point_threshould] = 255
        if contours(threshold_img)[5] != 0:
          m += 1
          integral_threshould += point_threshould
        if m > 150:
          break
    total_threshould = integral_threshould / m
    print(total_threshould) #決定された閾値

    cap = cv2.VideoCapture(filePath)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    n = 0
    old_n = 0
    old_y=0
    base_y=0
    area=0
    total_area=0
    width=0
    height=0
    k = 1
    pic_num = 1
    unitPixel = 150 #計測区間の大きさを設定
    maxPixel = 1080 #画像y座標の最大値を設定
    #if float(folderPath[-3]) >= 2.0:
    #  pixel_per_mm = 0.036
    #else:
    #  pixel_per_mm = 0.025
    pixel_per_mm = 0.036 #1ピクセルは何mm
    #pixel_per_mm = 0.023
    print(pixel_per_mm)
    #with open("/content/drive/MyDrive/data_analyses/" +str(video_path)+'諸データ'+'.csv','w') as dataFile:
    with open(str(fileName)+'諸データ'+'.csv','w') as dataFile:
     writer = csv.writer(dataFile, lineterminator='\n')
     writer.writerow([0,'フレーム数','火炎位置','火炎速度','火炎幅','火炎最大高さ','平均火炎高さ','面積'])


    #total_threshould = 8.5 #どうしても手動で閾値を決めたい時コメントアウトを外す

    while True:
         is_image,frame_img = cap.read()
         gray = cv2.cvtColor(frame_img, cv2.COLOR_RGB2GRAY)
         threshold_img = gray.copy()
         threshold_img[gray < total_threshould] = 0
         threshold_img[gray >= total_threshould] = 255
         contoursArray = contours(threshold_img)
         contoursNum = contoursArray[5]
         area = contoursArray[4]
         n += 1
         if contoursNum != 0 and (area * pixel_per_mm * pixel_per_mm) >0.1:
           old_n = n
           startPos = n
           break

    while True:
     while True: #画面内所定の座標に火炎が達するまで繰り返す
         is_image,frame_img = cap.read()
         gray = cv2.cvtColor(frame_img, cv2.COLOR_RGB2GRAY)
         threshold_img = gray.copy()
         threshold_img[gray < total_threshould] = 0
         threshold_img[gray >= total_threshould] = 255
         contoursNum = contours(threshold_img)[5]
         area = contours(threshold_img)[4]
         if contoursNum != 0 and (area * pixel_per_mm * pixel_per_mm) >0.1:
            contoursArray = contours(threshold_img)
            x_max = contoursArray[0]
            y_max = contoursArray[2] #火炎下端の位置を取得
            x_min = contoursArray[1]
            y_min = contoursArray[3]

         #print(x_max,x_min,y_max,y_min)
            if k > 1: #最初の区間は火炎の全景が見えていない可能性があるので計測しない 
                 width = (x_max - x_min)*pixel_per_mm
                 total_area = area * pixel_per_mm * pixel_per_mm
                 height = (y_max-y_min)*pixel_per_mm
                 ave_height = total_area / width
            else:
               width = "Nan"
               height = "Nan"
               ave_height = "Nan"
               total_area = "Nan"
            if y_max > (unitPixel * k) or y_max > maxPixel-10: #ここでループ判定
               #cv2.imwrite(R"/content/drive/MyDrive/data_analyses/frame_captures" + str(pic_num).zfill(digit) + ".jpg" , threshold_img) #GoogleDrive内の所定のフォルダに火炎画像を保存する
               pic_num += 1
               break
            n += 1
            #with open("/content/drive/MyDrive/data_analyses/" +str(video_path)+'諸データ'+'.csv','a') as dataFile:
            with open(str(fileName)+'諸データ'+'.csv','a') as dataFile:
               writer = csv.writer(dataFile, lineterminator='\n')
               writer.writerow([n-startPos,n,y_max,'',width,height,ave_height,total_area])
         else:
           y_max = "Nan"
           width = "Nan"
           total_area = "Nan"
           n += 1
           with open(str(fileName)+'諸データ'+'.csv','a') as dataFile:
             writer = csv.writer(dataFile, lineterminator='\n')
             writer.writerow([n-startPos,n,y_max,'',width,height,ave_height,total_area])
     velocity = (y_max-old_y)/((n-old_n)*(1/24))
     old_n = n
     k += 1
     old_y = y_max
     print(n,"y座標は",y_max,"　速度は",velocity*pixel_per_mm,"ミリ毎秒")
     #with open("/content/drive/MyDrive/data_analyses/" +str(video_path)+'諸データ'+'.csv','a') as dataFile:
     with open(str(fileName)+'諸データ'+'.csv','a') as dataFile:
           writer = csv.writer(dataFile, lineterminator='\n')
           writer.writerow([n-startPos,n,y_max,velocity*pixel_per_mm,width,height,ave_height,total_area])
     #print(total_area)
     #print(value)
     #print(k)
     if (k+1) * unitPixel >= maxPixel:
       break

    total_velocity = y_max / ((n-startPos) * (1/24))
    print("平均速度は",total_velocity*pixel_per_mm,"ミリ毎秒")

['NH4Cl-0.5']
2.0
0.036
73 y座標は 151 　速度は 2.718 ミリ毎秒
121 y座標は 301 　速度は 2.6999999999999997 ミリ毎秒
174 y座標は 453 　速度は 2.4778867924528303 ミリ毎秒
222 y座標は 601 　速度は 2.6639999999999997 ミリ毎秒
274 y座標は 753 　速度は 2.5255384615384617 ミリ毎秒
318 y座標は 901 　速度は 2.906181818181818 ミリ毎秒
平均速度は 2.6568737201365185 ミリ毎秒
2.0
0.036
98 y座標は 151 　速度は 1.73952 ミリ毎秒
146 y座標は 301 　速度は 2.6999999999999997 ミリ毎秒
193 y座標は 461 　速度は 2.941276595744681 ミリ毎秒
250 y座標は 602 　速度は 2.1372631578947368 ミリ毎秒
301 y座標は 753 　速度は 2.5581176470588236 ミリ毎秒
349 y座標は 905 　速度は 2.7359999999999998 ミリ毎秒
平均速度は 2.3985276073619635 ミリ毎秒


In [ ]:
folderList = []
p_temp = pathlib.Path('/content/drive/MyDrive/動画データ').glob(str(elementumName)+'*')
for p in p_temp:
  folderList.append(p.name)
for i in range(len(folderList)):
#for i in range(8,9):
  folderPath =  ('/content/drive/MyDrive/動画データ/'+str(folderList[i]))
  fileList = []
  q_temp = pathlib.Path(str(folderPath)).glob('*.mp4')
print(folderList)

['K2CO3-0.1', 'K2CO3-0.2', 'K2CO3-0.5', 'K2CO3-0.6', 'K2CO3-0.7', 'K2CO3-0.8', 'K2CO3-1.0', 'K2CO3-2.0', 'K2CO3-3.0']
